In [31]:
import pandas as pd
import numpy as np
import os
import sys
import math
import matplotlib.pyplot as plt

from scipy.stats import ttest_rel
from itertools import combinations
import statsmodels.stats.multitest as smm
from scipy.stats import f_oneway
from scipy import stats


In [32]:
# Assume the notebook is in the project root or a subfolder
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
mimic_emb_path = os.path.abspath(os.path.join(os.getcwd(), "..", "MIMIC_CXR_EMB"))

sys.path.append(project_root)
sys.path.append(mimic_emb_path)

from MIMIC_CXR_EMB.config_MIMIC import get_diseases, get_diseases_abbr

In [33]:
plt.rcParams['pdf.fonttype']=42 #ensures true fonte types are embedded in the pdf 
plt.rcParams['ps.fonttype']=42 # Applies the same setting for post script output

In [34]:
number_of_runs=5
significance_level=1.96 # for 95% ci
height = 6
font_size=11
rotation_degree =15

In [35]:
seed_19_sex = pd.read_csv("./Actual_TPR/Run_seed19_TPR_sex.csv",index_col=0)
seed_31_sex = pd.read_csv("./Actual_TPR/Run_seed31_TPR_sex.csv",index_col=0)
seed_38_sex = pd.read_csv("./Actual_TPR/Run_seed38_TPR_sex.csv",index_col=0)
seed_47_sex = pd.read_csv("./Actual_TPR/Run_seed47_TPR_sex.csv",index_col=0)
seed_77_sex = pd.read_csv("./Actual_TPR/Run_seed77_TPR_sex.csv",index_col=0)
seed_77_sex.head(5)

,diseases,TPR_M,TPR_F
0,Enlarged Cardiomediastinum,0.284710,0.183511
1,Cardiomegaly,0.673994,0.692411
2,Lung Opacity,0.770092,0.696099
3,Lung Lesion,0.210037,0.240458
4,Edema,0.563784,0.586787


In [36]:
seed_19_age = pd.read_csv("./Actual_TPR/Run_seed19_TPR_Age.csv",index_col=0)
seed_31_age = pd.read_csv("./Actual_TPR/Run_seed31_TPR_Age.csv",index_col=0)
seed_38_age = pd.read_csv("./Actual_TPR/Run_seed38_TPR_Age.csv",index_col=0)
seed_47_age = pd.read_csv("./Actual_TPR/Run_seed47_TPR_Age.csv",index_col=0)
seed_77_age = pd.read_csv("./Actual_TPR/Run_seed77_TPR_Age.csv",index_col=0)
seed_77_age.head(5)

,diseases,TPR_60-80,TPR_40-60,TPR_20-40,TPR_80-,TPR_0-20
0,Enlarged Cardiomediastinum,0.265306,0.205056,0.272000,NaN,0.555556
1,Cardiomegaly,0.710616,0.637488,0.638211,NaN,0.325000
2,Lung Opacity,0.774063,0.712230,0.648107,NaN,0.589286
3,Lung Lesion,0.248889,0.240786,0.120968,NaN,0.384615
4,Edema,0.560334,0.584588,0.517544,NaN,0.200000


In [37]:
seed_19_race = pd.read_csv("./Actual_TPR/Run_seed19_TPR_race.csv",index_col=0)
seed_31_race = pd.read_csv("./Actual_TPR/Run_seed31_TPR_race.csv",index_col=0)
seed_38_race = pd.read_csv("./Actual_TPR/Run_seed38_TPR_race.csv",index_col=0)
seed_47_race = pd.read_csv("./Actual_TPR/Run_seed47_TPR_race.csv",index_col=0)
seed_77_race = pd.read_csv("./Actual_TPR/Run_seed77_TPR_race.csv",index_col=0)
seed_77_race.head(3)

,diseases,TPR_White,TPR_Black,TPR_Hisp,TPR_Other,Gap_Asian,TPR_American
0,Enlarged Cardiomediastinum,0.273551,0.255102,0.196078,0.181818,0.375000,NaN
1,Cardiomegaly,0.694900,0.754771,0.752508,0.770213,0.620155,0.823529
2,Lung Opacity,0.775433,0.705549,0.725664,0.792627,0.708333,0.545455


In [38]:
seed_19_insurance = pd.read_csv("./Actual_TPR/Run_seed19_TPR_insurance.csv",index_col=0)
seed_31_insurance = pd.read_csv("./Actual_TPR/Run_seed31_TPR_insurance.csv",index_col=0)
seed_38_insurance = pd.read_csv("./Actual_TPR/Run_seed38_TPR_insurance.csv",index_col=0)
seed_47_insurance = pd.read_csv("./Actual_TPR/Run_seed47_TPR_insurance.csv",index_col=0)
seed_77_insurance = pd.read_csv("./Actual_TPR/Run_seed77_TPR_insurance.csv",index_col=0)
seed_77_insurance.head(3)

,diseases,TPR_Medicare,TPR_Other,TPR_Medicaid
0,Enlarged Cardiomediastinum,0.283951,0.266833,0.314286
1,Cardiomegaly,0.724058,0.706367,0.698514
2,Lung Opacity,0.787321,0.753744,0.711434


In [39]:
diseases=get_diseases()

diseases_filtered = [disease for disease in diseases if disease != 'No Finding']

diseases_abbr =get_diseases_abbr()


## 1. Gender

In [40]:
seed_19_sex.head(5)

,diseases,TPR_M,TPR_F
0,Enlarged Cardiomediastinum,0.318102,0.255319
1,Cardiomegaly,0.623220,0.621531
2,Lung Opacity,0.736502,0.703961
3,Lung Lesion,0.211896,0.244275
4,Edema,0.636757,0.618619


In [41]:
sex_dataframes = [seed_19_sex, seed_31_sex, seed_38_sex, seed_47_sex, seed_77_sex]
# Stack into a 3D array: (num_dfs, num_diseases, 2)
stacked = np.stack([df[['TPR_M', 'TPR_F']].values for df in sex_dataframes])

# Compute mean and standard error
mean_vals = stacked.mean(axis=0)
sem_vals = stats.sem(stacked, axis=0)

# Compute 95% CI half-width (using t-distribution for small sample size)
t_critical = stats.t.ppf(0.975, df=len(sex_dataframes) - 1)
ci_half_width = t_critical * sem_vals

# Construct final DataFrame
diseases = sex_dataframes[0]['diseases']
result_sex = pd.DataFrame({
    'diseases': diseases,
    'TPR_M_mean': mean_vals[:, 0],
    'TPR_M_CI': ci_half_width[:, 0],
    'TPR_F_mean': mean_vals[:, 1],
    'TPR_F_CI': ci_half_width[:, 1],
})

result_sex=result_sex[result_sex['diseases'].isin(diseases_filtered)]


In [42]:

# Extract just the Gap columns in the right order
gap_cols = ['TPR_M_mean','TPR_F_mean']

gap_df = result_sex[gap_cols]


# Store results
pairs = list(combinations(gap_cols, 2))
pvals = []
tstats = []

for a, b in pairs:
    t_stat, p_val = ttest_rel(gap_df[a], gap_df[b], nan_policy='omit')
    tstats.append(t_stat)
    pvals.append(p_val)

# Multiple comparisons correction (e.g., Bonferroni or FDR)
reject, pvals_corrected, _, _ = smm.multipletests(pvals, method='bonferroni')

# Report
for i, (a, b) in enumerate(pairs):
    print(f"{a} vs {b}: t={tstats[i]:.4f}, p={pvals[i]:.4f}, corrected_p={pvals_corrected[i]:.4f}, significant={reject[i]}")


TPR_M_mean vs TPR_F_mean: t=2.5227, p=0.0268, corrected_p=0.0268, significant=True


## 2. Age

In [46]:


# list of DataFrames with age-group TPRs
age_dataframes = [seed_19_age, seed_31_age, seed_38_age, seed_47_age, seed_77_age]

# Extract TPR columns (excluding 'diseases')
tpr_cols = [col for col in age_dataframes[0].columns if col != 'diseases']

# Stack into 3D array (n_dfs, n_diseases, n_cols)
stacked = np.stack([df[tpr_cols].values for df in age_dataframes])

# Compute mean and SEM across the seed dimension
mean_vals = np.nanmean(stacked, axis=0)
sem_vals = stats.sem(stacked, axis=0, nan_policy='omit')

# 95% CI half-width using t-distribution
t_critical = stats.t.ppf(0.975, df=len(age_dataframes) - 1)
ci_half_width = t_critical * sem_vals

# Build output DataFrame
diseases = age_dataframes[0]['diseases']
age_tpr_disp_df = pd.DataFrame({'diseases': diseases})

# Add each mean and CI column for each TPR
for i, col in enumerate(tpr_cols):
    age_tpr_disp_df[f'{col}_mean'] = np.round(mean_vals[:, i],3)
    age_tpr_disp_df[f'{col}_CI'] = np.round(ci_half_width[:, i],3)

age_tpr_disp_df=age_tpr_disp_df[age_tpr_disp_df['diseases'].isin(diseases_filtered)]
# Display or export result
age_tpr_disp_df.head(5)


C:\Users\gebre\AppData\Local\Temp\ipykernel_23052\68023387.py:11: RuntimeWarning: Mean of empty slice
  mean_vals = np.nanmean(stacked, axis=0)


,diseases,TPR_60-80_mean,TPR_60-80_CI,TPR_40-60_mean,TPR_40-60_CI,TPR_20-40_mean,TPR_20-40_CI,TPR_80-_mean,TPR_80-_CI,TPR_0-20_mean,TPR_0-20_CI
0,Enlarged Cardiomediastinum,0.278,0.059,0.189,0.042,0.269,0.038,NaN,NaN,0.511,0.076
1,Cardiomegaly,0.678,0.029,0.605,0.031,0.618,0.023,NaN,NaN,0.315,0.017
2,Lung Opacity,0.740,0.040,0.678,0.042,0.619,0.030,NaN,NaN,0.525,0.056
3,Lung Lesion,0.245,0.020,0.227,0.033,0.116,0.015,NaN,NaN,0.185,0.145
4,Edema,0.597,0.035,0.608,0.031,0.564,0.039,NaN,NaN,0.280,0.136


In [48]:
age_tpr_disp_df.shape

(13, 11)

In [49]:

# Extract just the Gap columns in the right order
gap_cols = [
    'TPR_60-80_mean','TPR_40-60_mean', 'TPR_20-40_mean','TPR_80-_mean', 'TPR_0-20_mean']

gap_df = age_tpr_disp_df[gap_cols]


# Store results
pairs = list(combinations(gap_cols, 2))
pvals = []
tstats = []

for a, b in pairs:
    t_stat, p_val = ttest_rel(gap_df[a], gap_df[b], nan_policy='omit')
    tstats.append(t_stat)
    pvals.append(p_val)

# Multiple comparisons correction (e.g., Bonferroni or FDR)
reject, pvals_corrected, _, _ = smm.multipletests(pvals, method='bonferroni')

# Report
for i, (a, b) in enumerate(pairs):
    print(f"{a} vs {b}: t={tstats[i]:.4f}, p={pvals[i]:.4f}, corrected_p={pvals_corrected[i]:.4f}, significant={reject[i]}")


TPR_60-80_mean vs TPR_40-60_mean: t=1.5519, p=0.1467, corrected_p=1.0000, significant=False
TPR_60-80_mean vs TPR_20-40_mean: t=1.1837, p=0.2595, corrected_p=1.0000, significant=False
TPR_60-80_mean vs TPR_80-_mean: t=nan, p=nan, corrected_p=nan, significant=False
TPR_60-80_mean vs TPR_0-20_mean: t=1.5503, p=0.1494, corrected_p=1.0000, significant=False
TPR_40-60_mean vs TPR_20-40_mean: t=0.5759, p=0.5753, corrected_p=1.0000, significant=False
TPR_40-60_mean vs TPR_80-_mean: t=nan, p=nan, corrected_p=nan, significant=False
TPR_40-60_mean vs TPR_0-20_mean: t=1.2402, p=0.2407, corrected_p=1.0000, significant=False
TPR_20-40_mean vs TPR_80-_mean: t=nan, p=nan, corrected_p=nan, significant=False
TPR_20-40_mean vs TPR_0-20_mean: t=1.4380, p=0.1783, corrected_p=1.0000, significant=False
TPR_80-_mean vs TPR_0-20_mean: t=nan, p=nan, corrected_p=nan, significant=False


In [50]:

df = age_tpr_disp_df

# Define the columns to use
gap_cols = ['TPR_60-80_mean','TPR_40-60_mean', 'TPR_20-40_mean', 'TPR_0-20_mean']

# Drop rows with missing values in those columns
gap_df = df[gap_cols].dropna()

# Extract values into 4 separate lists (one per group)
group_0_20 = gap_df['TPR_0-20_mean'].tolist()
group_20_40 = gap_df['TPR_20-40_mean'].tolist()
group_40_60 = gap_df['TPR_40-60_mean'].tolist()
group_60_80 = gap_df['TPR_60-80_mean'].tolist()


# Perform one-way ANOVA across the groups
f_stat, p_val = f_oneway(group_0_20, group_20_40, group_40_60, group_60_80)

# Show results

print(f"ANOVA result: F-statistic = {f_stat:.3f}, p-value = {p_val:.3f} significant={p_val < 0.05}")


ANOVA result: F-statistic = 0.398, p-value = 0.755 significant=False


## 3. Race

In [51]:

# List of race/ethnicity-based DataFrames
race_dataframes = [seed_19_race, seed_31_race, seed_38_race, seed_47_race, seed_77_race]

# Identify all TPR/Gap columns (excluding 'diseases')
tpr_cols = [col for col in race_dataframes[0].columns if col != 'diseases']

# Stack into a 3D array: (n_dfs, n_diseases, n_tpr_cols)
stacked = np.stack([df[tpr_cols].values for df in race_dataframes])

# Compute mean and standard error (ignoring NaNs)
mean_vals = np.nanmean(stacked, axis=0)
sem_vals = stats.sem(stacked, axis=0, nan_policy='omit')

# Compute 95% CI half-width using t-distribution
t_critical = stats.t.ppf(0.975, df=len(race_dataframes) - 1)
ci_half_width = t_critical * sem_vals

# Prepare final DataFrame
diseases = race_dataframes[0]['diseases']
race_tpr_disp_df = pd.DataFrame({'diseases': diseases})

# Fill in mean and CI columns
for i, col in enumerate(tpr_cols):
    race_tpr_disp_df[f'{col}_mean'] = np.round(mean_vals[:, i], 3)
    race_tpr_disp_df[f'{col}_CI'] = np.round(ci_half_width[:, i], 3)

race_tpr_disp_df=race_tpr_disp_df[race_tpr_disp_df['diseases'].isin(diseases_filtered)]
# Display result
race_tpr_disp_df.head(5)


C:\Users\gebre\AppData\Local\Temp\ipykernel_23052\316267013.py:11: RuntimeWarning: Mean of empty slice
  mean_vals = np.nanmean(stacked, axis=0)


,diseases,TPR_White_mean,TPR_White_CI,TPR_Black_mean,TPR_Black_CI,TPR_Hisp_mean,TPR_Hisp_CI,TPR_Other_mean,TPR_Other_CI,Gap_Asian_mean,Gap_Asian_CI,TPR_American_mean,TPR_American_CI
0,Enlarged Cardiomediastinum,0.263,0.062,0.269,0.081,0.220,0.027,0.194,0.021,0.400,0.088,NaN,NaN
1,Cardiomegaly,0.663,0.030,0.730,0.019,0.718,0.053,0.728,0.040,0.600,0.036,0.812,0.080
2,Lung Opacity,0.746,0.037,0.678,0.036,0.700,0.033,0.752,0.053,0.687,0.039,0.345,0.167
3,Lung Lesion,0.200,0.022,0.224,0.029,0.200,0.050,0.340,0.066,0.152,0.042,NaN,NaN
4,Edema,0.607,0.033,0.618,0.037,0.608,0.019,0.669,0.023,0.519,0.029,0.764,0.062


In [52]:

# Extract just the Gap columns in the right order
gap_cols = [
    'TPR_White_mean', 'TPR_Black_mean',
    'TPR_Hisp_mean', 'TPR_Other_mean','Gap_Asian_mean', 'TPR_American_mean']


gap_df = race_tpr_disp_df[gap_cols]

# Store results
pairs = list(combinations(gap_cols, 2))
pvals = []
tstats = []

for a, b in pairs:
    t_stat, p_val = ttest_rel(gap_df[a], gap_df[b], nan_policy='omit')
    tstats.append(t_stat)
    pvals.append(p_val)

# Multiple comparisons correction (e.g., Bonferroni or FDR)
reject, pvals_corrected, _, _ = smm.multipletests(pvals, method='bonferroni')

# Report
for i, (a, b) in enumerate(pairs):
    print(f"{a} vs {b}: t={tstats[i]:.4f}, p={pvals[i]:.4f}, corrected_p={pvals_corrected[i]:.4f}, significant={reject[i]}")


TPR_White_mean vs TPR_Black_mean: t=2.0989, p=0.0577, corrected_p=0.8649, significant=False
TPR_White_mean vs TPR_Hisp_mean: t=1.6790, p=0.1190, corrected_p=1.0000, significant=False
TPR_White_mean vs TPR_Other_mean: t=1.0015, p=0.3364, corrected_p=1.0000, significant=False
TPR_White_mean vs Gap_Asian_mean: t=0.0660, p=0.9485, corrected_p=1.0000, significant=False
TPR_White_mean vs TPR_American_mean: t=0.3885, p=0.7092, corrected_p=1.0000, significant=False
TPR_Black_mean vs TPR_Hisp_mean: t=0.5975, p=0.5613, corrected_p=1.0000, significant=False
TPR_Black_mean vs TPR_Other_mean: t=-0.0581, p=0.9546, corrected_p=1.0000, significant=False
TPR_Black_mean vs Gap_Asian_mean: t=-1.1665, p=0.2661, corrected_p=1.0000, significant=False
TPR_Black_mean vs TPR_American_mean: t=-0.0395, p=0.9696, corrected_p=1.0000, significant=False
TPR_Hisp_mean vs TPR_Other_mean: t=-0.4837, p=0.6373, corrected_p=1.0000, significant=False
TPR_Hisp_mean vs Gap_Asian_mean: t=-1.2183, p=0.2465, corrected_p=1.0000,

In [53]:

# Load your data
df = race_tpr_disp_df

# Define the columns to use
gap_cols = [
    'TPR_White_mean', 'TPR_Black_mean',
    'TPR_Hisp_mean', 'TPR_Other_mean','Gap_Asian_mean', 'TPR_American_mean']

# Drop rows with missing values in those columns
gap_df = df[gap_cols].dropna()

# Extract values into 4 separate lists (one per group)
group_W = gap_df['TPR_White_mean'].tolist()
group_B = gap_df['TPR_Black_mean'].tolist()
group_H = gap_df['TPR_Hisp_mean'].tolist()
group_O = gap_df['TPR_Other_mean'].tolist()
group_A = gap_df['Gap_Asian_mean'].tolist()
group_Am = gap_df['TPR_American_mean'].tolist()

# Perform one-way ANOVA across the groups
f_stat, p_val = f_oneway(group_W, group_B, group_H, group_O,group_A,group_Am)

print(f"ANOVA result: F-statistic = {f_stat:.3f}, p-value = {p_val:.3f} significant={p_val < 0.05}")


ANOVA result: F-statistic = 0.024, p-value = 1.000 significant=False


## 4.  Insurance

In [ ]:

# List of insurance-based TPR DataFrames
insurance_dataframes = [seed_19_insurance, seed_31_insurance, seed_38_insurance, seed_47_insurance, seed_77_insurance]

# Identify TPR columns (excluding 'diseases')
tpr_cols = [col for col in insurance_dataframes[0].columns if col != 'diseases']

# Stack into a 3D numpy array
stacked = np.stack([df[tpr_cols].values for df in insurance_dataframes])

# Compute means and SEM, ignoring NaNs
mean_vals = np.nanmean(stacked, axis=0)
sem_vals = stats.sem(stacked, axis=0, nan_policy='omit')

# Compute 95% CI half-width
t_critical = stats.t.ppf(0.975, df=len(insurance_dataframes) - 1)
ci_half_width = t_critical * sem_vals

# Construct result DataFrame
diseases = insurance_dataframes[0]['diseases']
insurance_tpr_disp_df = pd.DataFrame({'diseases': diseases})

# Add mean and CI columns
for i, col in enumerate(tpr_cols):
    insurance_tpr_disp_df[f'{col}_mean'] = np.round(mean_vals[:, i],3)
    insurance_tpr_disp_df[f'{col}_CI'] = np.round(ci_half_width[:, i],3)

insurance_tpr_disp_df=insurance_tpr_disp_df[insurance_tpr_disp_df['diseases'].isin(diseases_filtered)]
# Display result
insurance_tpr_disp_df.head(5)


,diseases,TPR_Medicare_mean,TPR_Medicare_CI,TPR_Other_mean,TPR_Other_CI,TPR_Medicaid_mean,TPR_Medicaid_CI
0,Enlarged Cardiomediastinum,0.270,0.060,0.264,0.049,0.314,0.038
1,Cardiomegaly,0.697,0.027,0.674,0.031,0.657,0.030
2,Lung Opacity,0.754,0.039,0.725,0.038,0.698,0.028
3,Lung Lesion,0.182,0.020,0.256,0.019,0.180,0.054
4,Edema,0.622,0.036,0.621,0.032,0.556,0.033


In [57]:

# Extract just the Gap columns in the right order
gap_cols = [
    'TPR_Medicare_mean', 'TPR_Other_mean', 'TPR_Medicaid_mean']


gap_df = insurance_tpr_disp_df[gap_cols]


# Store results
pairs = list(combinations(gap_cols, 2))
pvals = []
tstats = []

for a, b in pairs:
    t_stat, p_val = ttest_rel(gap_df[a], gap_df[b], nan_policy='omit')
    tstats.append(t_stat)
    pvals.append(p_val)

# Multiple comparisons correction (e.g., Bonferroni or FDR)
reject, pvals_corrected, _, _ = smm.multipletests(pvals, method='bonferroni')

# Report
for i, (a, b) in enumerate(pairs):
    print(f"{a} vs {b}: t={tstats[i]:.4f}, p={pvals[i]:.4f}, corrected_p={pvals_corrected[i]:.4f}, significant={reject[i]}")


TPR_Medicare_mean vs TPR_Other_mean: t=0.8540, p=0.4098, corrected_p=1.0000, significant=False
TPR_Medicare_mean vs TPR_Medicaid_mean: t=2.0880, p=0.0588, corrected_p=0.1763, significant=False
TPR_Other_mean vs TPR_Medicaid_mean: t=2.4376, p=0.0313, corrected_p=0.0939, significant=False


In [58]:

df = insurance_tpr_disp_df
# Define the columns to use
gap_cols = [
    'TPR_Medicare_mean', 'TPR_Other_mean', 'TPR_Medicaid_mean']

# Drop rows with missing values in those columns
gap_df = df[gap_cols].dropna()

# Extract values into 4 separate lists (one per group)
group_C = gap_df['TPR_Medicare_mean'].tolist()
group_O = gap_df['TPR_Other_mean'].tolist()
group_A = gap_df['TPR_Medicaid_mean'].tolist()

# Perform one-way ANOVA across the groups
f_stat, p_val = f_oneway(group_C, group_O, group_A)

print(f"ANOVA result: F-statistic = {f_stat:.3f}, p-value = {p_val:.3f} significant={p_val < 0.05}")


ANOVA result: F-statistic = 0.180, p-value = 0.836 significant=False
